In [ ]:
# Import Libraries

import tweepy 
import re
## Preprocessing 
import pandas as pd 
from collections import defaultdict

In [ ]:
# initialize api instance - Taori's Academic Research Account Creds

# ACCESS_TOKEN = "...."
# ACCESS_TOKEN_SECRET = "..."
# CONSUMER_KEY = "...."
# CONSUMER_SECRET = "..."
BEARER_TOKEN = "...."


In [ ]:
#Connect to Twitter through the API 

# auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) 
# auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET) 
# api = tweepy.API(auth,wait_on_rate_limit=True)

# To use tweepy v2 version, we need only bearer token
client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

In [ ]:
# Lat Long of 10 cities

NYC = {'lat': 40.712776, 'long': -74.005974}
LA = {'lat': 34.052235, 'long': -118.243683}
CHICAGO = {'lat': 41.878113, 'long': -87.629799}
BOSTON = {'lat': 42.360081, 'long': -71.058884}
NASHVILLE = {'lat': 36.162663, 'long': -86.781601}
DALLAS = {'lat': 32.776665, 'long': -96.796989}
SEATTLE = {'lat': 47.606209, 'long': -122.332069}
DENVER = {'lat': 39.739235, 'long': -104.990250}
NEW_ORLEANS = {'lat': 29.951065, 'long': -90.071533}
INDIANAPOLIS = {'lat': 39.768402, 'long': -86.158066}

RADIUS = "25mi"

# cities = {"NewYork": NYC, "LA": LA, "Chicago": CHICAGO}
cities = {"INDIANAPOLIS": INDIANAPOLIS}
# geo = ' '.join(str(item) for item in NYC.values()) + " " + RADIUS
# geo = "[{} {} {}]".format(NYC.get('long'), NYC.get('lat'), RADIUS)
# geo

In [ ]:
# out = client.search_all_tweets(query="(iphone15", max_results=500, start_time="2022-01-01T00:00:00.000000+00:00", expansions="geo.place_id,author_id", user_fields="public_metrics,verified", tweet_fields="created_at")
# out

In [ ]:
# MAX_TWEETS = 5000
# RADIUS = "25mi"
# COUNT_PER_PAGE = 100
# tweets_for_cities = defaultdict(list)

# def retrieve_and_store_tweets(search_query, lat_long, city_name):
#     tweets = tweepy.Cursor(api.search_tweets, q = search_query, geocode = lat_long, count = COUNT_PER_PAGE).items(MAX_TWEETS)
    
#     # Pulling information from tweets iterable object
#     # Add or remove tweet information you want in the below list comprehension
#     tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.favorite_count, tweet.user.screen_name, tweet.user.id_str, tweet.user.location, tweet.user.followers_count, tweet.coordinates, tweet.place] for tweet in tweets]

#     # Creation of dataframe from tweets_list
#     # Did not include column names to simplify code
#     tweets_df = pd.DataFrame(tweets_list)
    
#     tweets_for_cities[city_name].append(tweets_df)

In [ ]:
# List of keywords related to crime

sectors_and_topics = defaultdict(list)
sectors_and_topics["Crime"].extend(["Crime", "Theft", "Violence", "Gun Violence", "Abscond", "Harassmemt", 
                                            "SexualAbuse", "Assault", "Bribe", "Burglar", "Offense", "Unlawful", 
                                            "Forgery", "Illegal", "Gambling", "Violation", "Homicide", 
                                            "Imprisonment", "Trafficking", "Extrotion", "Manslaughter", "Criminal", 
                                            "Perjury", "Robbery", "Terrorism", "Terrorist"])

# retrieve_and_store_tweets(query, ','.join(str(item) for item in NYC.values()) + "," + RADIUS, "NewYork")

# api.search_full_archive(label="dev", query="crime", fromDate="201101010000", toDate="20211231000", maxResults=100)

In [ ]:
# Query building

query = " OR ".join(list(sectors_and_topics["Crime"]))
# point_radius = "[{} {} {}]".format(NYC.get('long'), NYC.get('lat'), RADIUS)
# query = "(" + query + ")" + " point_radius:" + point_radius
query

In [ ]:
# out = client.search_all_tweets(query=query, max_results=500, start_time="2022-01-17T00:00:00.000000+00:00", expansions="geo.place_id,author_id", user_fields="public_metrics,verified", tweet_fields="created_at")
# next_token = out.meta.get('next_token')
# tweet_text_1 = out.data[0].text
# out = client.search_all_tweets(query=query, max_results=500, start_time="2022-01-17T00:00:00.000000+00:00", expansions="geo.place_id,author_id", user_fields="public_metrics,verified", tweet_fields="created_at", next_token=next_token)
# out

In [ ]:
# Function to fetch all archival tweets with pagination

total_tweets_per_city_per_month = 50000
MAX_TWEETS = 500
tweets_per_page = total_tweets_per_city_per_month // MAX_TWEETS

def fetch_and_store_archival_tweets(city_name, query, start_time, end_time, month, year):
    next_token = ""

    # Name and path of the file where you want the Tweets written to
    file_name = "INDIANAPOLIS.csv"

    with open(file_name, 'a') as filehandle:
        for i in range(0, tweets_per_page):
            if not next_token:
                out = client.search_all_tweets(query=query, max_results=500, start_time=start_time, end_time=end_time, expansions="geo.place_id")
            else:
                out = client.search_all_tweets(query=query, max_results=500, start_time=start_time, end_time=end_time, expansions="geo.place_id", next_token=next_token)
            
            if (not out) or (not out.data):
                return
            
            for data in out.data:
                df = pd.DataFrame({"tweet": data.text, "tweet_id": data.id, "month": "{}-{}".format(month, year)}, index=[1])
                df.to_csv(file_name, mode='a', index=False, header=False)
#                 filehandle.write("{},{},{}".format(data.text, month, year))
            
            next_token = out.meta.get('next_token')
            if not next_token:
                return

def store_logs(start_time, end_time):
    file_name = "tweets_script_logs_INDIANAPOLIS.txt"
    
    with open(file_name, 'a') as filehandle:
        filehandle.write("{},{}\n".format(start_time, end_time))

In [ ]:
# Driver Function to loop for time range and fetch and store tweets

for city_name, geo in cities.items():
    point_radius = "[{} {} {}]".format(geo.get('long'), geo.get('lat'), RADIUS)
    query = "(" + query + ")" + " point_radius:" + point_radius
    for year in range(2011, 2022):
        for month in range(1, 13):
            if (year == 2013 and month >= 10) or (year >= 2014):
                month = "0{}".format(month) if month < 10 else month
                start_time = "{}-{}-01T00:00:00.000000+00:00".format(year, month)
                print("{}, {}".format(month, year))
                end_time = "{}-{}-28T00:00:00.000000+00:00".format(year, month)
                try:
                    fetch_and_store_archival_tweets(city_name, query, start_time, end_time, month, year)
                    store_logs(start_time, end_time)
                except Exception as e:
                    print("Start Time: {}, End Time: {}, Exception: {}".format(start_time, end_time, e))

In [ ]:
# Columns = ["Text", "CreatedAt", "TweetId", "FavoriteCount", "ScreenName", "UserId", "UserLocation", "UserFollowers", "UserCoordinate", "Place"]

# for city,list_of_tweets in tweets_for_cities.items():
#     topic_count = 0
#     for tweet_df in list_of_tweets:
#         if len(tweet_df) > 0:
#             tweets_df.columns = Columns
#             topic_count += 1
#             file_name = '~/Documents/Practice/TwitterData/{}/PublicSafety/Topic{}.csv'.format(city, topic_count)
#             tweet_df.to_csv(file_name, index = False, header=True)